# Duplication

Пример отчета по пересечению аудитории интернет-проектов (WEB-Index).

## Описание задачи

Посчитаем пересечение аудитории проектов:
- Vk.com
- Odnoklassniki.ru

Общие параметры:
- Период: Февраль 2020
- География: Россия 100 000+
- Население: 12+
- Фильтр по типу использования: нет, считаем по всем (Web Desktop, Web Mobile, App Online, App Offline)

Статистики:

- Reach
- Reach Column%
- Reach Row%


Для расчета потребуется сформировать три задания для Responsum:
 
- данные по пересечению аудитории проектов
- данные по аудитории проектов
- данные по аудитории всего Интернета
    

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from IPython.display import JSON
from bokeh.models import HoverTool
from bokeh.layouts import gridplot
import logging
import seaborn as sns
# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from mediascope_api.core import net as msnet
from mediascope_api.responsum import catalogs as rc
from mediascope_api.responsum import tasks as rt

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%I:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# pd.set_option("display.max_rows", 200)
# pd.set_option("display.max_colwidth", 50)
# pd.set_option("display.precision", 6)
output_notebook()

## Общие параметры для заданий

Для начала зададим общие параметры

In [ ]:
# задаем параметры
# выбираем тип установки mobile
facility = 'mobile' # возможные значения: 'desktop', 'mobile', 'desktop_pre'

# создаем объекты для работы с каталогами и заданиями,
# а так же загружаем каталоги
rcats = rc.ResponsumCats(facility)
rtask = rt.ResponsumTask(facility)

# задаем период расчета
date_from = '2020-02-01'
date_to = '2020-02-29'

# задаем типы пользования Интернетом
usetypes = rcats.get_usetype('all')

# проверяем, что значения параметров установлены верно
rtask.save_report_info(facility, date_from, date_to, usetypes)
rtask.show_report_info()
print(f"Объектов в media-каталоге: {rcats.holdings.shape[0]}")

### Получим ID проектов
Для построения отчета нам необходимо получить идентификаторы сайтов __Vk.com__ и __Odnoklassniki.ru__, для этого воспользуемся ноутбуком, в котором приведены примеры работы с [каталогами](catalogs.ipynb):

- Vk.com  site_id = 16571
- Odnoklassniki.ru site_id = 12808

## Задания
Перейдем к формированию заданий.


### Задание №1. Расчет пересечения аудитории проектов

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'dup'

# задаем медиа фильтр и duplication фильтр, в нашем случае это ID проекта VK.com и Odnoklassniki.ru
media_filter = "site = 16571"
dup_media_filter = "site = 12808"

# задаем фильтр, нас интересуют города России с населением 100 тыс. чел. и больше
demo_filter = "CITY_TYPE2 != 4"

# задаем список статистик для расчета
statistics = ["Reach"]

# указываем порядок группировки
structure =  {
    "usetype": False,
    "media": ["site"],
    "duplication": ["site"]
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_duplication_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, 
                                         media_filter=media_filter, dup_media_filter=dup_media_filter, 
                                         demo_filter=demo_filter, statistics=statistics, structure=structure)

# oтправляем задание на расчет и ждем выполнения
task_dup = rtask.wait_task(rtask.send_duplication_task(task_json))

# получаем результат
df_dup = rtask.result2table(rtask.get_result(task_dup), project_name=project_name)
# df_uni
df_dup

Посчитаем дополнительные отчеты, необходимые для расчета Reach Row% и Reach Column%.


### Задание №2. Расчет Total для проектов

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'total_project'

# задаем фильтр по медиа
media_filter = "site = 16571 OR site = 12808"

# задаем фильтр, нас интересуют города России с населением 100 тыс. чел. и больше
demo_filter = "CITY_TYPE2 != 4"

# задаем список статистик для расчета
statistics = ["Reach"]

# указываем порядок разбивки по сайтам
structure =  {
    "usetype": False,
    "media": ["site"]
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total_prj = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total_prj

### Задание №3. Расчет Total Internet

Расчет данных по всем проектам займет достаточно много времени у Responsum, поэтому воспользуемся технической страницей WEB-Index, в которой учитывается вся аудитория.
Техническая страница WEB-Index __site_id = 101__

Важно учитывать в случае расчета статистики OTS:
- если считать с ограничением по Технической странице в медиа-фильтре, то полученная статистика OTS будет показывать общее количество заходов в Интернет за указанный период;
- если считать без ограничений в медиа-фильтре (media_filter = None), то полученная статистика OTS будет показывать общее количество загрузок страниц интернет-проектов, содержащихся в вашем медиа-каталоге, за указанный период;
- статистика OTS корректна для работы с десктопными данными и данными по рекламным кампаниям.

In [ ]:
# задаем название проекта для отображения в DataFrame
project_name = 'total'

# задаем техническую страницу WEB-Index в качестве медиа фильтра
media_filter = 'site = 101'

# задаем фильтр, нас интересуют города России с населением 100 тыс. чел. и больше
demo_filter = "CITY_TYPE2 != 4"

# задаем список статистик для расчета
statistics = ["Reach"]

# указываем порядок группировки, в нашем случае ее нет
structure =  {
    "usetype": False
  }

# формируем из заданных параметров задание для Responsum в формате JSON
task_json = rtask.build_audience_task(task_name=project_name, facility=facility, date_from=date_from, date_to=date_to, usetype_filter=usetypes, media_filter=media_filter, demo_filter=demo_filter, statistics=statistics, structure=structure)

# отправляем задание на расчет и ждем выполнения
task_audience = rtask.wait_task(rtask.send_audience_task(task_json))

# получаем результат
df_total = rtask.result2table(rtask.get_result(task_audience), project_name=project_name)
df_total

### Формируем итоговую таблицу

Объединим полученные результаты и посчитаем доли.

Для этого воспользуемся методом библиотеки Mediascope _calc_duplication_row_col_.

Для этого метода на вход передаются три DataFrame:

- данные по пересечению аудитории проектов
- Total для проектов
- Total Internet

In [ ]:
df_duplication = rtask.calc_duplication_row_col(df_dup, df_total_prj, df_total)
df_duplication

#### Округлим полученные значения до второго знака
Воспользуемся стандартным методом библиотеки pandas

In [ ]:
df_duplication[['rnd_reach', 'rnd_prc_reach']] = df_duplication[['media_stat_reach', 'stat_prc_reach']].round(2)
df_duplication

## Экспорт в Excel

In [ ]:
writer = pd.ExcelWriter(rtask.get_excel_filename('duplication-VK-OK'))
df_info = rtask.get_report_info()
df_duplication.to_excel(writer, 'Report')
df_info.to_excel(writer, 'Info', index=False)
writer.save()